In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('IMDB Dataset.csv')

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
# Review Example
data['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [5]:
data.shape

(50000, 2)

In [6]:
data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

## Text Preprocessing
* Steps
1) Encoding target column if textual
2) Remove HTML tags
3) Remove Special characters
4) Converting everything to lower case
5) Removing stop words
6) Stemming

## Encoding

In [7]:
data['sentiment'].replace({'positive' : 1, 'negative' : 0}, inplace= True)

In [8]:
data.head(1)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1


## Remove HTML tags

In [9]:
# Example
clean = re.compile('<.*?>')
re.sub(clean, '', data.iloc[2].review)

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [10]:
# Main operation
def clear_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

data['review'] = data['review'].apply(clear_html)

## Remove special characters

In [11]:
def remove_special(text):
    x = ''
    
    for i in text:
        if i.isalnum():
            x = x + i
        else:
            x = x + ' '
    return x

In [12]:
# Example
remove_special('I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.')

'I thought this was a wonderful way to spend time on a too hot summer weekend  sitting in the air conditioned theater and watching a light hearted comedy  The plot is simplistic  but the dialogue is witty and the characters are likable  even the well bread suspected serial killer   While some may be disappointed when they realize this is not Match Point 2  Risk Addiction  I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love This was the most I d laughed at one of Woody s comedies in years  dare I say a decade    While I ve never been impressed with Scarlet Johanson  in this she managed to tone down her  sexy  image and jumped right into a average  but spirited young woman This may not be the crown jewel of his career  but it was wittier than  Devil Wears Prada  and more interesting than  Superman  a great comedy to go see with friends '

In [13]:
# Main operation
data['review'] = data['review'].apply(remove_special)

## Converting everything into lower case

In [14]:
def convert_lower(text):
    return text.lower()

data['review'] = data['review'].apply(convert_lower)

In [15]:
# Output
data['review'][2]

'i thought this was a wonderful way to spend time on a too hot summer weekend  sitting in the air conditioned theater and watching a light hearted comedy  the plot is simplistic  but the dialogue is witty and the characters are likable  even the well bread suspected serial killer   while some may be disappointed when they realize this is not match point 2  risk addiction  i thought it was proof that woody allen is still fully in control of the style many of us have grown to love this was the most i d laughed at one of woody s comedies in years  dare i say a decade    while i ve never been impressed with scarlet johanson  in this she managed to tone down her  sexy  image and jumped right into a average  but spirited young woman this may not be the crown jewel of his career  but it was wittier than  devil wears prada  and more interesting than  superman  a great comedy to go see with friends '

## Removing stop words

In [16]:
stopwords_list = stopwords.words('english')
print(stopwords_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
def remove_stopwords(text):
    x = []
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y = x[:]
    x.clear()
    return y

data['review'] = data['review'].apply(remove_stopwords)

In [18]:
data.head()

,review,sentiment
0,"[one, reviewers, mentioned, watching, 1, oz, e...",1
1,"[wonderful, little, production, filming, techn...",1
2,"[thought, wonderful, way, spend, time, hot, su...",1
3,"[basically, family, little, boy, jake, thinks,...",0
4,"[petter, mattei, love, time, money, visually, ...",1


## Stemming

In [19]:
ps = PorterStemmer()

y = []

def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z = y[:]
    y.clear()
    return z

data['review'] = data['review'].apply(stem_words)

## List to string

In [20]:
def list_to_string(text):
    return ' '.join(text)

data['review'] = data['review'].apply(list_to_string)

In [21]:
data['review'][0]

'one review mention watch 1 oz episod hook right exactli happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side'

## Vectorization

In [22]:
cv = CountVectorizer(max_features=5000)

X = cv.fit_transform(data['review']).toarray()

In [23]:
X.shape

(50000, 5000)

In [24]:
y = data.iloc[:,-1].values

In [25]:
y.shape

(50000,)

## Train Test Split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40000, 5000), (10000, 5000), (40000,), (10000,))

## Model training

In [28]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()

In [29]:
clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)

BernoulliNB()

In [30]:
y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)

In [34]:
print('GaussianNB:')
print('Accuracy Score: ',accuracy_score(y_test,y_pred1))
print('Confusion Matrix:\n ',confusion_matrix(y_test,y_pred1))
print('Classification Report:\n ',classification_report(y_test,y_pred1),'-----------------------------------------------------')

print('MultinomialNB:')
print('Accuracy Score: ',accuracy_score(y_test,y_pred2))
print('Confusion Matrix:\n ',confusion_matrix(y_test,y_pred2))
print('Classification Report:\n ',classification_report(y_test,y_pred2),'-----------------------------------------------------')

print('BernoulliNB:')
print('Accuracy Score: ',accuracy_score(y_test,y_pred3))
print('Confusion Matrix:\n ',confusion_matrix(y_test,y_pred3))
print('Classification Report:\n ',classification_report(y_test,y_pred3),'-----------------------------------------------------')

GaussianNB:
Accuracy Score:  0.7148
Confusion Matrix:
  [[4222  739]
 [2113 2926]]
Classification Report:
                precision    recall  f1-score   support

           0       0.67      0.85      0.75      4961
           1       0.80      0.58      0.67      5039

    accuracy                           0.71     10000
   macro avg       0.73      0.72      0.71     10000
weighted avg       0.73      0.71      0.71     10000
 -----------------------------------------------------
MultinomialNB:
Accuracy Score:  0.8471
Confusion Matrix:
  [[4220  741]
 [ 788 4251]]
Classification Report:
                precision    recall  f1-score   support

           0       0.84      0.85      0.85      4961
           1       0.85      0.84      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000
 -----------------------------------------------------
BernoulliNB:
Accura

# Thank You!